In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test_labels.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv


In [2]:
#---------------------------------------------------
# PHẦN 1: CÀI ĐẶT VÀ CHUẨN BỊ DỮ LIỆU
#---------------------------------------------------

# 1.1. Cài đặt thư viện AutoGluon
# Lệnh này cần thiết vì AutoGluon không có sẵn trên Kaggle
!pip install autogluon --quiet

import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

print("="*50)
print("PHẦN 1: ĐANG TẢI VÀ CHUẨN BỊ DỮ LIỆU")
print("="*50)

# 1.2. Khai báo đường dẫn và tải dữ liệu
path_toxic_comment = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv'
path_unintended_bias = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv'

try:
    df_toxic = pd.read_csv(path_toxic_comment)
    df_bias = pd.read_csv(path_unintended_bias)

    # Ghép nối và chuẩn hóa dữ liệu như các bước trước
    df_toxic_subset = df_toxic[['comment_text', 'toxic']]
    df_bias_subset = df_bias[['comment_text', 'toxic']]
    full_train_df = pd.concat([df_toxic_subset, df_bias_subset], ignore_index=True)
    full_train_df['toxic'] = full_train_df['toxic'].apply(lambda x: 1 if x >= 0.5 else 0)
    
    # Để chạy nhanh hơn cho ví dụ này, chúng ta sẽ lấy một mẫu nhỏ
    # BỎ CHÚ THÍCH DÒNG DƯỚI ĐÂY NẾU BẠN MUỐN CHẠY TRÊN TOÀN BỘ DỮ LIỆU (sẽ mất rất nhiều thời gian)
    full_train_df = full_train_df.sample(n=100000, random_state=42)
    
    print(f"Đã tạo DataFrame training tổng hợp với {len(full_train_df)} mẫu.")

except FileNotFoundError as e:
    print(f"\nLỖI: Không tìm thấy file training. Quy trình dừng lại.")
    print(f"Chi tiết lỗi: {e}")
    exit()

#---------------------------------------------------
# PHẦN 2: CHIA DỮ LIỆU (80% TRAIN, 20% TEST)
#---------------------------------------------------
print("\n" + "="*50)
print("PHẦN 2: CHIA DỮ LIỆU THÀNH TẬP TRAIN VÀ TEST")
print("="*50)

# Chia dữ liệu thành 80% train và 20% test (để đánh giá cuối cùng)
# stratify=full_train_df['toxic'] rất quan trọng để đảm bảo tỉ lệ nhãn 'toxic'
# là như nhau trong cả hai tập train và test.
train_data, test_data = train_test_split(
    full_train_df,
    test_size=0.2,
    random_state=42,
    stratify=full_train_df['toxic']
)

print(f"Kích thước tập Train: {train_data.shape}")
print(f"Kích thước tập Test: {test_data.shape}")
print(f"Phân phối nhãn trong tập Train:\n{train_data['toxic'].value_counts(normalize=True)}")
print(f"Phân phối nhãn trong tập Test:\n{test_data['toxic'].value_counts(normalize=True)}")


#---------------------------------------------------
# PHẦN 3: HUẤN LUYỆN VỚI AUTOGLUON
#---------------------------------------------------
print("\n" + "="*50)
print("PHẦN 3: BẮT ĐẦU HUẤN LUYỆN VỚI AUTOGLUON")
print("="*50)

# Khởi tạo TabularPredictor
# AutoGluon sẽ tự động xử lý cột 'comment_text' như một đặc trưng văn bản
predictor = TabularPredictor(
    label='toxic',                # Cột mục tiêu cần dự đoán
    problem_type='binary',        # Loại bài toán: phân loại nhị phân
    eval_metric='roc_auc',        # Thước đo để tối ưu, phù hợp với cuộc thi
    path='./ag_models_toxic'      # Thư mục để lưu các mô hình đã huấn luyện
)

# Huấn luyện mô hình
# AutoGluon sẽ thử nhiều mô hình khác nhau và kết hợp chúng lại
# time_limit là giới hạn thời gian huấn luyện (tính bằng giây)
# presets='best_quality' để có kết quả tốt nhất, bạn có thể dùng 'high_quality' hoặc 'medium_quality' để nhanh hơn
predictor.fit(
    train_data,
    time_limit=1800, # Giới hạn thời gian 30 phút. Tăng lên để có kết quả tốt hơn.
    presets='high_quality'
)

#---------------------------------------------------
# PHẦN 4: ĐÁNH GIÁ MÔ HÌNH TRÊN TẬP TEST
#---------------------------------------------------
print("\n" + "="*50)
print("PHẦN 4: ĐÁNH GIÁ HIỆU SUẤT MÔ HÌNH")
print("="*50)

# Xem bảng xếp hạng các mô hình đã được huấn luyện
print("Bảng xếp hạng các mô hình (đánh giá trên tập validation nội bộ của AutoGluon):")
leaderboard = predictor.leaderboard(silent=True)
print(leaderboard)

# Đánh giá hiệu suất trên tập test 20% mà chúng ta đã tách ra
print("\nĐánh giá trên tập test (20% dữ liệu giữ lại):")
performance = predictor.evaluate(test_data)
print(performance)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.5/454.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.4/382.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 13.4 MB/s eta 0:

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       27.54 GB / 31.35 GB (87.8%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input da

Đã tạo DataFrame training tổng hợp với 100000 mẫu.

PHẦN 2: CHIA DỮ LIỆU THÀNH TẬP TRAIN VÀ TEST
Kích thước tập Train: (80000, 2)
Kích thước tập Test: (20000, 2)
Phân phối nhãn trong tập Train:
toxic
0    0.9191
1    0.0809
Name: proportion, dtype: float64
Phân phối nhãn trong tập Test:
toxic
0    0.9191
1    0.0809
Name: proportion, dtype: float64

PHẦN 3: BẮT ĐẦU HUẤN LUYỆN VỚI AUTOGLUON


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-07-02 03:34:52,830	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
		Context path: "/kaggle/working/ag_models_toxic/ds_sub_fit/sub_fit_ho"
(_dystack pid=311) Running DyStack sub-fit ...
(_dystack pid=311) Beginning AutoGluon training ... Time limit = 443s
(_dystack pid=311) AutoGluon will save models to "/kaggle/working/ag_models_toxic/ds_sub_fit/sub_fit_ho"
(_dystack pid=311) Train Data Rows:    71111
(_dystack pid=311) Train Data Columns: 1
(_dystack pid=311) Label Column:       toxic
(_dystack pid=311) Problem Type:       binary
(_dystack pid=311) Preprocessing data ...
(_dystack pid=311) Selected class <--> label mapping:  class 1 = 1, class 0 = 0
(_dystack pid=311) Using Feature Generators to preprocess the data ...
(_dystack pid=311


PHẦN 4: ĐÁNH GIÁ HIỆU SUẤT MÔ HÌNH
Bảng xếp hạng các mô hình (đánh giá trên tập validation nội bộ của AutoGluon):
                      model  score_val eval_metric  pred_time_val    fit_time  \
0       WeightedEnsemble_L2   0.878422     roc_auc      26.672686  584.572414   
1       WeightedEnsemble_L3   0.878422     roc_auc      26.673401  584.574960   
2         LightGBMXT_BAG_L1   0.875186     roc_auc      15.078219  339.478667   
3           LightGBM_BAG_L1   0.869766     roc_auc      11.580819  244.361953   
4  WeightedEnsemble_L3_FULL        NaN     roc_auc            NaN   86.874781   
5  WeightedEnsemble_L2_FULL        NaN     roc_auc            NaN   86.872235   
6      LightGBM_BAG_L1_FULL        NaN     roc_auc            NaN   39.095640   
7    LightGBMXT_BAG_L1_FULL        NaN     roc_auc            NaN   47.044801   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.013647           0.731794            2      False   
1          

In [5]:
predictor.leaderboard(test_data, extra_metrics = ['accuracy', 'log_loss', 'f1_macro', 'f1_weighted', 'roc_auc'])

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,model,score_test,accuracy,log_loss,f1_macro,f1_weighted,roc_auc,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3_FULL,0.880810,0.93940,-0.183294,0.719990,0.927750,0.880810,NaN,roc_auc,9.669240,NaN,86.874781,0.002003,NaN,0.734340,3,True,8
1,WeightedEnsemble_L2_FULL,0.880810,0.93940,-0.183294,0.719990,0.927750,0.880810,NaN,roc_auc,9.670047,NaN,86.872235,0.002809,NaN,0.731794,2,True,7
2,LightGBMXT_BAG_L1_FULL,0.879606,0.93950,-0.181599,0.727571,0.928976,0.879606,NaN,roc_auc,5.081077,NaN,47.044801,5.081077,NaN,47.044801,1,True,5
3,LightGBM_BAG_L1_FULL,0.868408,0.93715,-0.190716,0.699108,0.923434,0.868408,NaN,roc_auc,4.586161,NaN,39.095640,4.586161,NaN,39.095640,1,True,6
4,WeightedEnsemble_L2,NaN,NaN,NaN,NaN,NaN,NaN,0.878422,roc_auc,NaN,26.672686,584.572414,NaN,0.013647,0.731794,2,False,3
5,WeightedEnsemble_L3,NaN,NaN,NaN,NaN,NaN,NaN,0.878422,roc_auc,NaN,26.673401,584.574960,NaN,0.014363,0.734340,3,False,4
6,LightGBMXT_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,0.875186,roc_auc,NaN,15.078219,339.478667,NaN,15.078219,339.478667,1,False,1
7,LightGBM_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,0.869766,roc_auc,NaN,11.580819,244.361953,NaN,11.580819,244.361953,1,False,2
